In [1]:
%config c.use_jedi = False

In [2]:
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [ ]:
import glob, os
from io import StringIO
import collections, numpy
import pandas as pd
import numpy as np
import plotly.graph_objs as go

def get_words(x):
	temp_result = x.split('\t\t')[0]
	if temp_result.find('\t') > -1:
			temp_result = x.split('\t')[0]
	if temp_result.find(' ') > -1:
			temp_result = x.split()[0]
	return temp_result
def get_phonemes(x):
	temp_result = ""
	if x.find('\t\t') > -1:
			temp_result = x.split('\t\t')[1]
	if (temp_result.find('\t') > -1) or (len(temp_result) == 0 and x.find('\t') > -1):
			temp_result = x.split('\t')[1]
	if (temp_result.find(' ') > -1) or (len(temp_result) == 0 and x.find(' ') > -1):
			temp_result = x.split()[1:]
	return temp_result
	
def process_data(name, normalized=True, reference_dict=None):
	perdas = 0
	if name == 'cmu':
		df = pd.read_csv('../data/external/cmudict.dict', sep='\t', header=None, names=['lines'])
		df.lines = df.lines.str.replace('\d', '') #Dígitos representam apenas entonação https://en.wikipedia.org/wiki/ARPABET
		df['words'] = df.lines.str.split().str[0]
		df['phonemes']  = df.lines.str.split().str[1:]
		df = df.drop(labels=df[df.lines.str.contains('#')].index).reset_index(drop=True)
	if name == 'dict_br':
		df = pd.read_csv('../data/external/dictionary.dic', sep=';', header=None, names=['lines'])
		df.lines = df.lines.str.replace('sp$', '').str.replace('sp $', '')
		df = (
				df
				.drop(labels=df[df.lines.str.contains('\[\]')].index)
				.reset_index(drop=True)
		)
		df['words'] = df.lines.apply(get_words)
		df['phonemes'] = df.lines.apply(get_phonemes)
		df = (
			df
			.drop(labels=df[df.phonemes.apply(lambda x : 'sil' in x)].index)
			.reset_index(drop=True)
		)
	if name == 'laps' or 'const' or 'cell':
		myBigString = ""
		if name == 'laps':
			for file in glob.glob("../data/external/LapsBM1.4/*/*.txt", recursive=True):
				myBigString += open(file).read() + ' '
		if name == 'const':
			for file in glob.glob("../data/external/constituicaoBR/*.txt", recursive=True):
				myBigString += open(file).read() + ' '
		if name == 'cell':
			import pysrt
			for file in glob.glob("../data/external/Cellbit/originals/*.srt", recursive=True):
				subs = pysrt.open(file)
				myBigString += subs.text
			myBigString = myBigString.replace('\n', '').lower()
		t = collections.Counter(numpy.array(myBigString.split()))
		del myBigString #save memory
		df = pd.DataFrame(list(zip(t.keys(), t.values())), columns=['words', 'ocorrencias'])
		try:
			antes = len(df)
			df = df.merge(reference_dict, on='words')
			depois = len(df)
			perdas = 1.0 - (depois/antes)
		except:
			raise Exception('Deve definir um dicionário para processar')
		df.phonemes = df.phonemes * df.ocorrencias #multiplica as ocorrencias



	unique, counts = np.unique(
			np.hstack(
					df.phonemes.values
			),
			return_counts=True)

	phonemes = pd.DataFrame(list(zip(unique, counts)), columns=['phoneme', 'cmu_dict'])
	
	if name == 'cmu':
		phonemes = (
				phonemes
				.merge(conversor, left_on='phoneme', right_on='two_letter')
				.sort_values(by='cmu_dict', ascending=False)
		)
	else:
		phonemes = (
	    phonemes
	    .merge(conversor.drop_duplicates(subset='one_letter'),left_on='phoneme', right_on='one_letter')
	    .sort_values(by='cmu_dict', ascending=False)
	    )

	data = go.Bar(
						x=phonemes.ipa,
						y=(
								100 *
								phonemes.cmu_dict / 
								phonemes.cmu_dict.sum()
						) if normalized else phonemes.cmu_dict,
						name=name
			)

	return df, phonemes, data, perdas


## Descrição CMU

In [4]:
df = pd.read_csv('../data/external/cmudict.dict', sep='\t', header=None, names=['lines'])
df.lines = df.lines.str.replace('\d', '') #Dígitos representam apenas entonação https://en.wikipedia.org/wiki/ARPABET
df['words'] = df.lines.str.split().str[0]
df['phonemes']  = df.lines.str.split().str[1:]
df = df.drop(labels=df[df.lines.str.contains('#')].index).reset_index(drop=True)

In [10]:
(
    df
    .assign(word_len = df.words.apply(len), phonemes_len = df.phonemes.apply(len))
    .describe()
)

,word_len,phonemes_len
count,135083.000000,135083.000000
mean,7.662622,6.384652
std,2.383080,2.136503
min,1.000000,1.000000
25%,6.000000,5.000000
50%,7.000000,6.000000
75%,9.000000,8.000000
max,28.000000,28.000000


## Descrição Dict_BR

In [13]:
def get_words(x):
	temp_result = x.split('\t\t')[0]
	if temp_result.find('\t') > -1:
			temp_result = x.split('\t')[0]
	if temp_result.find(' ') > -1:
			temp_result = x.split()[0]
	return temp_result

def get_phonemes(x):
	temp_result = ""
	if x.find('\t\t') > -1:
			temp_result = x.split('\t\t')[1]
	if (temp_result.find('\t') > -1) or (len(temp_result) == 0 and x.find('\t') > -1):
			temp_result = x.split('\t')[1]
	if (temp_result.find(' ') > -1) or (len(temp_result) == 0 and x.find(' ') > -1):
			temp_result = x.split()[1:]
	return temp_result
df = pd.read_csv('../data/external/dictionary.dic', sep=';', header=None, names=['lines'])
df.lines = df.lines.str.replace('sp$', '').str.replace('sp $', '')
df = (
        df
        .drop(labels=df[df.lines.str.contains('\[\]')].index)
        .reset_index(drop=True)
)
df['words'] = df.lines.apply(get_words)
df['phonemes'] = df.lines.apply(get_phonemes)
df = (
    df
    .drop(labels=df[df.phonemes.apply(lambda x : 'sil' in x)].index)
    .reset_index(drop=True)
)

In [15]:
(
    df
    .assign(word_len = df.words.apply(len), phonemes_len = df.phonemes.apply(len))
    .describe()
)

,word_len,phonemes_len
count,65318.000000,65318.000000
mean,7.914189,7.372838
std,2.590464,2.443116
min,1.000000,1.000000
25%,6.000000,6.000000
50%,8.000000,7.000000
75%,10.000000,9.000000
max,23.000000,22.000000


## Descrição LAPS

### Apenas Texto

In [21]:
import glob, os
from io import StringIO
import collections
myBigString = ""
for file in glob.glob("../data/external/constituicaoBR/*.txt", recursive=True):
    myBigString += open(file).read() + ' '
t = collections.Counter(np.array(myBigString.split()))
del myBigString #save memory
df = pd.DataFrame(list(zip(t.keys(), t.values())), columns=['words', 'ocorrencias'])

In [24]:
(
    df
    .assign(word_len = df.words.apply(len))
    .describe()
)

,ocorrencias,word_len
count,5329.000000,5329.00000
mean,13.099456,8.70107
std,102.709031,2.73274
min,1.000000,1.00000
25%,1.000000,7.00000
50%,2.000000,9.00000
75%,5.000000,10.00000
max,4412.000000,21.00000


### Apenas Áudio

In [29]:
import librosa
lenghts = []
for file in glob.glob("../data/external/constituicaoBR/*.wav", recursive=True):
    lenghts.append(librosa.get_duration(filename=file))
df = pd.DataFrame(lenghts, columns=['lenghts'])

In [30]:
(
    df
    .describe()
)

,lenghts
count,1255.000000
mean,25.738601
std,5.989798
min,3.353878
25%,22.568095
50%,26.781995
75%,29.940998
max,42.579592


### Texto e Áudio

In [35]:
todos = []
for file in glob.glob("../data/external/constituicaoBR/*.txt", recursive=True)[:10]:
    todos.append([file[32:-4], open(file).read(), librosa.get_duration(filename=file[:-4]+'.wav')])
df = pd.DataFrame(todos, columns=['filename', 'text', 'lenght'])
df.head()

,filename,text,lenght
0,dt058a,artigo cinquenta e oito os benefícios de pres...,27.886984
1,art231b,são terras tradicionalmente ocupadas pelos ín...,23.276009
2,art105i,parágrafo único funcionarão junto ao superior...,33.867982
3,art022g,inciso vinte e dois competência da polícia fe...,22.789025
4,art103-d,será dada ciência ao poder competente para a ...,30.537143


In [95]:
import scipy
df = (df
    .assign(word_sep=df.text.str.split().apply(lambda x : [len(y) for y in x]).apply(scipy.stats.describe).apply(list))
)

In [104]:
(
    df
    .assign(pps=df.word_sep.apply(lambda x : x[0])/df.lenght, 
           avg_palavras=df.word_sep.apply(lambda x : x[2]),
           std_palavras=df.word_sep.apply(lambda x : x[3]),
           total_palavras=df.word_sep.apply(lambda x : x[0]))
    .apply({
        'lenght':[np.sum, np.mean, np.std],
        'total_palavras':[np.mean, np.std],
        'pps': [np.mean, np.std],
        'avg_palavras':[np.mean, np.std],
        'std_palavras':[np.mean, np.std]
    })
)

,lenght,total_palavras,pps,avg_palavras,std_palavras
mean,28.797374,60.700000,2.112487,5.405661,12.013307
std,6.166879,16.138635,0.405732,0.431429,2.322666
sum,287.973741,NaN,NaN,NaN,NaN


## Descrição do Cellbit

In [9]:
import pysrt
import glob, os
import pandas as pd
import librosa
todos = []
for file in glob.glob("../data/external/Cellbit/wavs/*.wav", recursive=True):
    todos.append([file[32:-4], librosa.get_duration(filename=file)])
df = pd.DataFrame(todos, columns=['filename', 'lenght'])
df.head()

,filename,lenght
0,013_00093,3.54
1,011_00124,2.00
2,001_00232,1.98
3,019_00033,2.18
4,020_00036,1.54


In [10]:
df.lenght.mean()

2.8799794662348326

In [11]:
df.lenght.std()

4.189267680072042